## Descarga y tratamiento de datos

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from scipy.optimize import minimize
from scipy.stats import jarque_bera
import cvxpy as cp
from cvxopt import matrix, solvers
import requests
import json
import urllib3
urllib3.disable_warnings()
from datetime import datetime
import pytz

El primer paso práctico en la construcción de nuestro portafolio es la adquisición de datos históricos de los activos en los que deseamos invertir.

Utilizamos el módulo requests de Python para manejar sesiones y cookies, permitiéndonos mantener una conexión persistente durante nuestras solicitudes de datos a un servicio web.

Esto es esencial para asegurar la continuidad y la integridad de la sesión de usuario durante la descarga de datos desde plataformas que requieren autenticación o mantenimiento de estado.

In [2]:
# Genero una sesion para mantener las cookies.
s = requests.session()
a = s.get('https://open.bymadata.com.ar/#/dashboard', verify=False)

In [3]:
a.text

'<!DOCTYPE html>\n<html>\n  <head>\n    <meta charset="utf-8" />\n\n    <title>BYMADATA - Cotizaciones en Tiempo Real | BYMA Bolsas y Mercados Argentinos</title>\n\n    <meta name="description" content="BYMA te acerca la informaciÃ³n mÃ¡s relevante del Mercado de Capitales argentino en un solo lugar." />\n\n    <meta name="facebook-domain-verification" content="x9zbmyu0rbwsxvklczmdu6ujdbq5s8" />\n\n    <!-- base url -->\n    <base href="/" />\n\n    <meta\n      name="viewport"\n      content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=no"\n    />\n    <link\n      href="https://fonts.googleapis.com/icon?family=Material+Icons"\n      rel="stylesheet"\n    />\n    <!-- #CSS Links -->\n    <!-- Basic Styles -->\n    <link\n      rel="stylesheet"\n      type="text/css"\n      media="screen"\n      href="assets/css/bootstrap.min.css"\n    />\n    <link\n      rel="stylesheet"\n      type="text/css"\n      media="screen"\n      href="assets/css/font-awesome.min.

### Extracción de datos para múltiples activos

El proceso es similar para múltiples activos, pero se realiza dentro de un bucle que itera a través de una lista de tickers.

Almacenamos cada DataFrame en un diccionario que facilita el acceso y la manipulación posterior.

Este método nos permite manejar eficientemente varios activos, fundamental para la construcción de un portafolio diversificado.

In [4]:
# Lista de tickers
tickers = [
    "ALUA", "BBAR", "BMA", "BYMA", "CEPU", "COME", "CRES",
    "EDN", "GFVA", "GGAL", "IRSA", "LOMA", "MIRG", "PAMP",
    "SUP", "TECO2", "TGNO4", "TGSU2", "TRAN", "TXAR",
    "VALO", "YPFD"
]

# Parámetros de tiempo
inicio = int(datetime(2020, 1, 1, 0, 0).timestamp())
hoy = datetime.now(pytz.timezone('America/Argentina/Buenos_Aires')).timestamp()

# Diccionario para almacenar los DataFrames de cada ticker
dfs = {}

for ticker in tickers:
    params = (
        ('symbol', f'{ticker} CDO'), ## Nombre de especie y plazo (CDO / 24HS / 48HS)
        ('resolution', 'D'),  ## puede ser D - Diario, S - Semanal, M - Mensual,
        ('from', str(inicio)), ## Unix epoch, se deberia poder armar como int(datetime.datetime(AÑO,MES,DIA,HORA,MINUTOS).timestamp())
        ('to', str(int(hoy))), ## Unix epoch, se deberia poder armar como int(datetime.datetime(AÑO,MES,DIA,HORA,MINUTOS).timestamp())
    )

    response = requests.get('https://open.bymadata.com.ar/vanoms-be-core/rest/api/bymadata/free/chart/historical-series/history', params=params, verify=False)

    if response.status_code == 200:
        data = json.loads(response.text)
        df = pd.DataFrame(data)
        df['t'] = pd.to_datetime(df['t'], unit='s').dt.normalize()
        df.rename({'s':'status', 't':'timestamp', 'c':'close', 'o':'open', 'h':'high', 'l':'low', 'v':'volume'}, axis = 1, inplace=True)
        dfs[ticker] = df
    else:
        print(f"No se pudo obtener datos para {ticker}")

# Puedes acceder a cada DataFrame con dfs['TICKER']
dfs

{'ALUA':     status  timestamp   close   open   high     low  volume
 0       ok 2022-08-01  137.25  140.5  144.0  135.75   16941
 1       ok 2022-08-02  139.00  136.0  142.0  136.00   43185
 2       ok 2022-08-03  137.50  140.0  144.0  134.00   19449
 3       ok 2022-08-04  135.25  140.0  140.0  132.25   17123
 4       ok 2022-08-05  134.25  135.0  139.0  134.25    7705
 ..     ...        ...     ...    ...    ...     ...     ...
 517     ok 2024-09-13  912.00  917.0  920.0  892.00  187036
 518     ok 2024-09-16  926.00  919.0  935.0  913.00   53631
 519     ok 2024-09-17  917.00  930.0  930.0  901.00   34006
 520     ok 2024-09-18  898.00  928.0  928.0  887.00   42384
 521     ok 2024-09-19  884.00  907.0  920.0  866.00  107009
 
 [522 rows x 7 columns],
 'BBAR':     status  timestamp   close    open    high     low  volume
 0       ok 2022-08-01   255.0   257.1   270.0   255.0    1275
 1       ok 2022-08-02   251.2   247.0   258.0   246.0    4913
 2       ok 2022-08-03   257.6   260

### Consolidación de datos

Finalmente, consolidamos los datos de todos los activos en un único DataFrame.

Esto nos permite observar y analizar los precios de cierre de todos los activos en una matriz temporal, donde las filas representan fechas y las columnas representan cada activo.

La visualización y análisis de esta matriz serán cruciales para decisiones subsecuentes en la asignación de activos y la evaluación del riesgo y retorno del portafolio.

In [5]:
# Concatena todos los DataFrames en uno solo
datos = pd.concat(dfs.values(), keys=dfs.keys())

# Restablece el índice para que 'timestamp' sea una columna y no un índice
datos = datos.reset_index()

# Renombra las columnas para que sea más claro
datos.rename(columns={'level_0': 'Ticker', 'timestamp': 'Date'}, inplace=True)

# Pivotea los datos para obtener los precios de cierre con fechas en filas y tickers en columnas
datos = datos.pivot(index='Date', columns='Ticker', values='close')

# Convierte todos los valores a numérico, por si acaso algunos fueron leídos como strings
for column in datos.columns:
    datos[column] = pd.to_numeric(datos[column], errors='coerce')

# Convierte la columna de índice 'Date' a datetime si no lo está ya
datos.index = pd.to_datetime(datos.index)

# Ordena el DataFrame por la fecha de manera descendente (las fechas más recientes primero)
datos.sort_index(ascending=False, inplace=True)

datos = datos.reset_index()

# Ahora pivot_data es similar a lo que se muestra en tu segunda imagen
datos

Ticker,Date,ALUA,BBAR,BMA,BYMA,CEPU,COME,CRES,EDN,GGAL,...,LOMA,MIRG,PAMP,TECO2,TGNO4,TGSU2,TRAN,TXAR,VALO,YPFD
0,2024-09-19,884.00,4895.0,9130.00,340.50,1270.00,280.50,1125.00,1535.0,5730.00,...,2200.00,21325.0,3050.00,2075.00,3280.0,5140.00,1770.0,848.00,317.5,29575.00
1,2024-09-18,898.00,4750.0,8800.00,332.00,1270.00,281.00,1115.00,1500.0,5680.00,...,2145.00,20525.0,2930.00,1955.00,3220.0,5100.00,1770.0,831.00,315.5,29125.00
2,2024-09-17,917.00,4745.0,8590.00,329.50,1275.00,281.50,1140.00,1520.0,5560.00,...,2090.00,21775.0,2915.00,2045.00,3265.0,5200.00,1850.0,861.00,319.5,29525.00
3,2024-09-16,926.00,4710.0,8330.00,336.50,1310.00,280.50,1135.00,1520.0,5510.00,...,2045.00,21825.0,2950.00,2085.00,3270.0,5290.00,1845.0,863.00,320.0,29900.00
4,2024-09-13,912.00,4720.0,8370.00,336.50,1300.00,279.00,1130.00,1535.0,5700.00,...,2010.00,21350.0,2950.00,2000.00,3250.0,5240.00,1830.0,832.00,320.0,29125.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
517,2022-08-05,134.25,255.0,374.85,194.50,113.05,15.85,177.60,100.5,222.65,...,345.35,3947.0,278.25,255.00,129.0,339.95,84.0,160.00,56.6,1097.00
518,2022-08-04,135.25,252.3,378.80,197.00,109.55,15.15,180.30,100.4,223.05,...,334.95,4075.0,274.30,255.00,115.5,333.65,75.8,159.00,56.9,1081.55
519,2022-08-03,137.50,257.6,382.00,194.00,110.25,15.00,176.45,97.2,226.25,...,338.75,3880.0,279.30,255.80,113.0,331.05,73.9,157.25,57.0,1090.65
520,2022-08-02,139.00,251.2,390.00,199.50,109.00,15.10,167.25,92.9,224.10,...,333.00,3873.5,263.90,250.15,111.5,325.55,74.9,163.00,55.5,1074.70


In [ ]:
datos.to_csv('Datos.csv', index=False)

In [6]:
# Mostrar información básica sobre el conjunto de datos
informacion_datos = datos.info()

# Mostrar las primeras filas del conjunto de datos para entender su estructura
primeras_filas_datos = datos.head()

informacion_datos, primeras_filas_datos

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 522 entries, 0 to 521
Data columns (total 21 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    522 non-null    datetime64[ns]
 1   ALUA    522 non-null    float64       
 2   BBAR    522 non-null    float64       
 3   BMA     522 non-null    float64       
 4   BYMA    522 non-null    float64       
 5   CEPU    522 non-null    float64       
 6   COME    522 non-null    float64       
 7   CRES    522 non-null    float64       
 8   EDN     522 non-null    float64       
 9   GGAL    522 non-null    float64       
 10  IRSA    509 non-null    float64       
 11  LOMA    522 non-null    float64       
 12  MIRG    520 non-null    float64       
 13  PAMP    522 non-null    float64       
 14  TECO2   522 non-null    float64       
 15  TGNO4   522 non-null    float64       
 16  TGSU2   522 non-null    float64       
 17  TRAN    522 non-null    float64       
 18  TXAR    52

(None,
 Ticker       Date   ALUA    BBAR     BMA   BYMA    CEPU   COME    CRES  \
 0      2024-09-19  884.0  4895.0  9130.0  340.5  1270.0  280.5  1125.0   
 1      2024-09-18  898.0  4750.0  8800.0  332.0  1270.0  281.0  1115.0   
 2      2024-09-17  917.0  4745.0  8590.0  329.5  1275.0  281.5  1140.0   
 3      2024-09-16  926.0  4710.0  8330.0  336.5  1310.0  280.5  1135.0   
 4      2024-09-13  912.0  4720.0  8370.0  336.5  1300.0  279.0  1130.0   
 
 Ticker     EDN    GGAL  ...    LOMA     MIRG    PAMP   TECO2   TGNO4   TGSU2  \
 0       1535.0  5730.0  ...  2200.0  21325.0  3050.0  2075.0  3280.0  5140.0   
 1       1500.0  5680.0  ...  2145.0  20525.0  2930.0  1955.0  3220.0  5100.0   
 2       1520.0  5560.0  ...  2090.0  21775.0  2915.0  2045.0  3265.0  5200.0   
 3       1520.0  5510.0  ...  2045.0  21825.0  2950.0  2085.0  3270.0  5290.0   
 4       1535.0  5700.0  ...  2010.0  21350.0  2950.0  2000.0  3250.0  5240.0   
 
 Ticker    TRAN   TXAR   VALO     YPFD  
 0       177

El conjunto de datos tiene 516 registros y 20 columnas correspondientes a diferentes acciones del panel líder del Merval. Algunas columnas como IRSA, MIRG, y VALO tienen valores nulos (menos de 516 valores no nulos), lo que indica que necesito manejar esos valores ausentes como parte del proceso de limpieza de datos.

El primer paso será manejar los valores nulos presentes en algunas de las columnas, imputándolos con la media de la respectiva serie de datos. Esto garantizará la integridad de los datos y evitará posibles sesgos en el análisis.

Luego, procederé a identificar y eliminar cualquier duplicado en las filas del conjunto de datos, ya que la duplicación de registros podría distorsionar los resultados del análisis. 